In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3`
import $ivy.`sh.almond::almond-spark:0.6.0`

import org.apache.spark._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

val spark = NotebookSparkSession
      .builder()
      .config("spark.sql.join.preferSortMergeJoin", false)
      .config("spark.sql.shuffle.partitions", 64)
      .master("local[*]")
      .getOrCreate()

import spark.implicits._

Loading spark-stubs
Getting spark JARs


log4j:WARN No appenders could be found for logger (org.eclipse.jetty.util.log).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
22/02/12 13:14:14 INFO SparkContext: Running Spark version 2.4.3
22/02/12 13:14:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/02/12 13:14:14 INFO SparkContext: Submitted application: 8d5a3dd6-376a-48b2-9c20-b2c15b71ba63
22/02/12 13:14:14 INFO SecurityManager: Changing view acls to: oscar
22/02/12 13:14:14 INFO SecurityManager: Changing modify acls to: oscar
22/02/12 13:14:14 INFO SecurityManager: Changing view acls groups to: 
22/02/12 13:14:14 INFO SecurityManager: Changing modify acls groups to: 
22/02/12 13:14:14 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(oscar); groups with view permissions: Set(); users  with modify permissions: Set(oscar); groups with modify permissions: Set()
22/02/12 13:14:15 INFO Utils: Successfully started service 'sparkDrive

22/02/12 13:14:15 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/mainargs_2.12/0.1.4/mainargs_2.12-0.1.4-sources.jar at spark://netrunner:43159/jars/mainargs_2.12-0.1.4-sources.jar with timestamp 1644668055373
22/02/12 13:14:15 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/mainargs_2.12/0.1.4/mainargs_2.12-0.1.4.jar at spark://netrunner:43159/jars/mainargs_2.12-0.1.4.jar with timestamp 1644668055373
22/02/12 13:14:15 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/os-lib_2.12/0.7.1/os-lib_2.12-0.7.1-sources.jar at spark://netrunner:43159/jars/os-lib_2.12-0.7.1-sources.jar with timestamp 1644668055373
22/02/12 13:14:15 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/os-lib_2.12/0.7.1/os-lib_2.12-0.7.1.jar at spark://netrunner:43159/jars/os-lib_2.12-0.7.1.j

22/02/12 13:14:15 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/github/alexarchambault/argonaut-shapeless_6.2_2.12/1.2.0-M11/argonaut-shapeless_6.2_2.12-1.2.0-M11.jar at spark://netrunner:43159/jars/argonaut-shapeless_6.2_2.12-1.2.0-M11.jar with timestamp 1644668055379
22/02/12 13:14:15 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/org/eclipse/jetty/jetty-server/9.4.19.v20190610/jetty-server-9.4.19.v20190610.jar at spark://netrunner:43159/jars/jetty-server-9.4.19.v20190610.jar with timestamp 1644668055379
22/02/12 13:14:15 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/io/argonaut/argonaut_2.12/6.2.3/argonaut_2.12-6.2.3.jar at spark://netrunner:43159/jars/argonaut_2.12-6.2.3.jar with timestamp 1644668055379
22/02/12 13:14:15 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/chuusai/shapeless_2.

Spark UI

import $ivy.$                                  

import $ivy.$                              


import org.apache.spark._

import org.apache.spark.sql.{functions => func, _}

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._


import org.slf4j.LoggerFactory

import org.apache.log4j.{Level, Logger}


spark: SparkSession = org.apache.spark.sql.SparkSession@3b47a6e5
import spark.implicits._


defined function run

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Window


In [5]:
spark = SparkSession.builder\
      .config("spark.sql.shuffle.partitions", 64)\
      .master("local[*]")\
      .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/12 18:05:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/02/12 18:05:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
races = spark.read.format("csv")\
    .option("header", "true")\
    .option("sep", ",")\
    .load("../data/races.csv")\
    .select("raceId", "year")

In [11]:
seasonWindow = Window.partitionBy("year")
driverRaceWindow = Window.partitionBy("driverId", "raceId")
raceDriverLapWindow = Window.partitionBy("driverId", "raceId").orderBy("lap")

In [16]:
overtakes = spark.read.format("csv")\
    .option("header", "true")\
    .option("sep", ",")\
    .load("../data/lap_times.csv")\
    .withColumn("position", F.col("position").cast(T.IntegerType()))\
    .withColumn("lap", F.col("lap").cast(T.IntegerType()))\
    .join(races, "raceId")\
    .withColumn("positionNextLap", F.lead(F.col("position"), 1).over(raceDriverLapWindow))\
    .withColumn("positionsGainedLap", F.when(F.col("positionNextLap") < F.col("position") , F.abs(F.col("position") - F.col("positionNextLap"))).otherwise(0))\
    .groupBy("raceId", "driverId")\
    .agg(F.sum(F.col("positionsGainedLap")).alias("positionsGained"), F.first(F.col("year")).alias("year"))\
    .groupBy("year")\
    .agg(F.sum(F.col("positionsGained")).alias("positionsGainedSeason"))\
    .withColumn("rankPositionsGained", F.rank().over(Window.orderBy(F.col("positionsGainedSeason").desc())))

In [20]:
leadersTroughoutSeason = spark.read.format("csv")\
    .option("header", "true")\
    .option("sep", ",")\
    .load("../data/driver_standings.csv")\
    .join(races, "raceId")\
    .where(F.col("position") == 1)\
    .dropDuplicates(["driverId", "position", "year"])\
    .groupBy("year")\
    .agg(F.approx_count_distinct(F.col("driverId")).alias("distinctLeaders"))\
    .withColumn("rankDistinctLeaders", F.rank().over(Window.orderBy(F.col("distinctLeaders").desc())))

In [21]:
winnersTroughoutSeason = spark.read.format("csv")\
    .option("header", "true")\
    .option("sep", ",")\
    .load("../data/results.csv")\
    .join(races, "raceId")\
    .where(F.col("position") == 1)\
    .dropDuplicates(["driverId", "position", "year"])\
    .groupBy("year")\
    .agg(F.approx_count_distinct(F.col("driverId")).alias("distinctWinners"))\
    .withColumn("rankDistinctWinners", F.rank().over(Window.orderBy(F.col("distinctWinners").desc())))

In [26]:
val averageRank = udf((cols: Seq[Int]) => {cols.sum / cols.size}: Double)
spark.udf.register("averageRank", averageRank)

averageRank: expressions.UserDefinedFunction = UserDefinedFunction(
  ammonite.$sess.cmd25$Helper$$Lambda$6499/1616635210@394c4368,
  DoubleType,
  Some(List(ArrayType(IntegerType, false)))
)
res25_1: expressions.UserDefinedFunction = UserDefinedFunction(
  ammonite.$sess.cmd25$Helper$$Lambda$6499/1616635210@394c4368,
  DoubleType,
  Some(List(ArrayType(IntegerType, false)))
)

In [46]:
def averageRank(cols):
    return sum(cols) / len(cols)

In [48]:
averageRank = F.udf(averageRank)

In [ ]:
overtakes\
    .join(leadersTroughoutSeason, "year", "inner")\
    .join(winnersTroughoutSeason, "year", "inner")\
    .withColumn("avgRank", (F.col("rankDistinctWinners") + F.col("rankDistinctLeaders") + F.col("rankPositionsGained") / 3))\
    .withColumn("overallRank", F.rank().over(Window.orderBy("avgRank")))\
    .drop("rankDistinctWinners", "rankDistinctLeaders", "rankPositionsGained", "avgRank")\
    .sort("overallRank")\
    .show()

Usar la UDF definida arriba da resultados erróneos. Por ello se suman las columnas y se divide entre 3. Esto es menos flexible ya que si quisiera hacer la media del contenido de 6 columnas tendría que hacerlo a mano. Para solucionarlo también se puede castear la columna avgRank a Integer. Por defecto debe ser que las UDF devuelven un String en Python.

In [67]:
overtakes\
    .join(leadersTroughoutSeason, "year", "inner")\
    .join(winnersTroughoutSeason, "year", "inner")\
    .withColumn("avgRank", averageRank(F.array(F.col("rankDistinctWinners"), F.col("rankDistinctLeaders"), F.col("rankPositionsGained"))).cast(T.IntegerType()))\
    .withColumn("overallRank", F.rank().over(Window.orderBy("avgRank")))\
    .drop("rankDistinctWinners", "rankDistinctLeaders", "rankPositionsGained", "avgRank")\
    .sort("overallRank")\
    .show()

22/02/12 19:29:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/02/12 19:29:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/02/12 19:29:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/02/12 19:29:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/02/12 19:29:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/02/12 19:29:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/02/12 1

+----+---------------------+---------------+---------------+-----------+
|year|positionsGainedSeason|distinctLeaders|distinctWinners|overallRank|
+----+---------------------+---------------+---------------+-----------+
|2012|                 5077|              4|              8|          1|
|2008|                 3123|              4|              7|          2|
|2003|                 2955|              3|              8|          3|
|1997|                 2642|              3|              6|          4|
|2010|                 2747|              6|              5|          5|
|1999|                 2151|              3|              6|          6|
|2021|                 2901|              2|              6|          7|
|2013|                 4697|              2|              5|          8|
|2019|                 3201|              2|              5|          9|
|2007|                 2974|              3|              4|         10|
|2020|                 2612|              2|       

22/02/12 19:29:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/02/12 19:29:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/02/12 19:29:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
